In [1]:
import pandas as pd
import numpy as np
import math
from random import randrange

In [2]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [3]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity

In [4]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [5]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [6]:
class Rule:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

    def match(self, example):
        val = example[self.column]
        try:
            if is_numeric(val):
                return val >= self.value
            else:
                return val == self.value
        except TypeError:
            return 'nan'


In [7]:
def build_stump(rows, rule):
    true_rows, false_rows = [], []
    for row in rows:
        if isinstance(row[rule.column],float) and math.isnan(row[rule.column]):
            continue
        if rule.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [8]:
def find_split(rows,n_features):
    best_gain = 0
    best_rule = None
    current_uncertainty = gini(rows)
    total_features = len(rows[0]) - 1
    
    if n_features is None:
        feature_list = range(total_features)
    else:
        feature_list = np.random.choice(range(total_features), n_features, replace=False)

    for col in feature_list:
        values = set([row[col] for row in rows])
        for val in values:
            rule = Rule(col, val)
            true_rows, false_rows = build_stump(rows, rule)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_rule = gain, rule

    return best_gain, best_rule

In [9]:
class Leaf:
    def __init__(self, rows):
        val = class_counts(rows)
        itemMax = max(val.items(), key=lambda x : x[1])
        self.predictions = itemMax[0]

In [10]:
class Decision_Node:
    def __init__(self,rule,true_branch,false_branch):
        self.rule = rule
        self.true_branch = true_branch
        self.false_branch = false_branch

In [11]:
def build_tree(rows, max_depth = 10, n_features = None, depth = 0):
    gain, rule = find_split(rows, n_features)
    if gain == 0 or depth >= max_depth:
        return Leaf(rows)
    true_rows, false_rows = build_stump(rows, rule)
    true_branch = build_tree(true_rows, max_depth, n_features, depth+1)
    false_branch = build_tree(false_rows, max_depth, n_features, depth+1)
    return Decision_Node(rule, true_branch, false_branch)

In [12]:
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [13]:
def build_random_forest(rows, max_depth, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(rows, sample_size)
        tree = build_tree(sample, max_depth, n_features)
        trees.append(tree)
    return trees

In [14]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "└── Predict", node.predictions)
        return
    print (spacing + str(node.rule))

    print (spacing + '├── True:')
    print_tree(node.true_branch, spacing + "|  ")

    print (spacing + '├── False:')
    print_tree(node.false_branch, spacing + "|  ")

In [15]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions

    if node.rule.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [16]:
def bagging_predict(row, trees):
    predictions = [classify(row, tree) for tree in trees]
    return max(set(predictions), key=predictions.count)

In [17]:
#testing only 

df = pd.read_csv("Datasets/working_csv7.csv")
Y = df["P. Habitable Class"]
X = df.drop("P. Habitable Class",axis=1)
header = list(df.columns)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

X_train_list = X_train.values.tolist()
X_test_list = X_test.values.tolist()
Y_train_list = Y_train.values.tolist()
Y_test_list = Y_test.values.tolist()
    
training_data = list(map(lambda x:x[0]+[x[1]],zip(X_train_list,Y_train_list)))
testing_data = list(map(lambda x:x[0]+[x[1]],zip(X_test_list,Y_test_list)))

max_depth = 10
sample_size = 1.0
n_trees = 15
n_features = int(math.sqrt(len(training_data[0])-1))

my_forest = build_random_forest(training_data,max_depth,sample_size,n_trees,n_features)

y_pred = []
y_test = []

for i,row in enumerate(testing_data):
    val = bagging_predict(row,my_forest)
    
    y_test.append(row[-1])
    y_pred.append(val)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3 0 1]
 [0 7 1]
 [0 0 5]]
               precision    recall  f1-score   support

   mesoplanet       1.00      0.75      0.86         4
non-habitable       1.00      0.88      0.93         8
psychroplanet       0.71      1.00      0.83         5

     accuracy                           0.88        17
    macro avg       0.90      0.88      0.87        17
 weighted avg       0.92      0.88      0.89        17



In [19]:
#evaluate model on all 500 datasets

from sklearn.model_selection import train_test_split
from functools import reduce
scores = []
for i in range(500):
    df = pd.read_csv("Datasets/working_csv"+str(i)+".csv")
    Y = df["P. Habitable Class"]
    X = df.drop("P. Habitable Class",axis=1)

    header = list(df.columns)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

    X_train_list = X_train.values.tolist()
    X_test_list = X_test.values.tolist()
    Y_train_list = Y_train.values.tolist()
    Y_test_list = Y_test.values.tolist()

    training_data = list(map(lambda x:x[0]+[x[1]],zip(X_train_list,Y_train_list)))
    testing_data = list(map(lambda x:x[0]+[x[1]],zip(X_test_list,Y_test_list)))
    
    max_depth = 10
    sample_size = 1.0
    n_trees = 50
    n_features = int(math.sqrt(len(training_data[0])-1))

    my_forest = build_random_forest(training_data,max_depth,sample_size,n_trees,n_features)
    
    count = 0
    for i,row in enumerate(testing_data):
        val = bagging_predict(row,my_forest)
        if val == Y_test_list[i]:
            count += 1
    scores.append([count/len(Y_test_list),count,len(Y_test_list)])
test_df = pd.DataFrame(scores, columns=["Accuracy", "right", "total"])
print(reduce(lambda x,y: [(x[1]+y[1])/(x[2]+y[2]),(x[1]+y[1]),(x[2]+y[2])],scores))
test_df

[0.9494117647058824, 8070, 8500]


,Accuracy,right,total
0,1.000000,17,17
1,0.941176,16,17
2,0.941176,16,17
3,1.000000,17,17
4,1.000000,17,17
5,0.941176,16,17
6,0.882353,15,17
7,0.882353,15,17
8,0.823529,14,17
9,1.000000,17,17


In [21]:
for my_tree in my_forest:
    print_tree(my_tree)
    print()

Is S. Mag from Planet >= -26.4?
├── True:
|  Is P. Name == GJ 422 b?
|  ├── True:
|  |  └── Predict mesoplanet
|  ├── False:
|  |  └── Predict psychroplanet
├── False:
|  Is P. HZC >= 2.45?
|  ├── True:
|  |  └── Predict non-habitable
|  ├── False:
|  |  Is P. Habitable >= 1?
|  |  ├── True:
|  |  |  └── Predict mesoplanet
|  |  ├── False:
|  |  |  └── Predict non-habitable

Is P. SPH >= 0.17?
├── True:
|  └── Predict mesoplanet
├── False:
|  Is P. SFlux Max (EU) >= 3.203192?
|  ├── True:
|  |  └── Predict non-habitable
|  ├── False:
|  |  └── Predict psychroplanet

Is P. SFlux Max (EU) >= 0.7529996999999999?
├── True:
|  Is P. SFlux Min (EU) >= 3.203191?
|  ├── True:
|  |  └── Predict non-habitable
|  ├── False:
|  |  Is S. RA (hrs) >= 1.7344?
|  |  ├── True:
|  |  |  └── Predict mesoplanet
|  |  ├── False:
|  |  |  └── Predict psychroplanet
├── False:
|  └── Predict psychroplanet

Is P. Mag >= -18.9?
├── True:
|  Is P. SFlux Min (EU) >= 0.6584886?
|  ├── True:
|  |  └── Predict mesop